# Cassandra & CQL Guide

We will use python 3.8 + Datastax driver and this jupyter notebook to run the codes (for convenience)

![title](./img/Cassandra_Tutorials_02.png)

![title](./img/Cassandra_Tutorials_03.png)

![title](./img/Cassandra_Tutorials_04.png)

![title](./img/Cassandra_Tutorials_05.png)

### Initalization

Run `cassandra` on cmd / terminal (single node)

In [1]:
from cassandra.cluster import Cluster

cluster = Cluster(port=9042)
session = cluster.connect()

In [2]:
session.execute("select release_version from system.local").all()

[Row(release_version='3.0.27')]

In [3]:
# Drop all incase you run this again
session.execute("TRUNCATE TABLE snu.student")
session.execute("DROP TABLE snu.student")
session.execute("DROP KEYSPACE snu")

In [4]:
session.execute("SELECT keyspace_name FROM system_schema.keyspaces").all()

[Row(keyspace_name='system_auth'),
 Row(keyspace_name='system_schema'),
 Row(keyspace_name='system_distributed'),
 Row(keyspace_name='system'),
 Row(keyspace_name='system_traces')]

In [5]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address    Load       Tokens       Owns (effective)  Host ID                               Rack
UN  127.0.0.1  191.77 KB  256          100.0%            dbc99623-ab6e-4807-9b9d-68f141db7221  rack1



### Basic queries

#### Create Keyspace

In [6]:
# Keyspace = Database
session.execute(
'''
CREATE KEYSPACE snu 
    WITH REPLICATION = { 
        'class' : 'SimpleStrategy', 
        'replication_factor' : 1
        } 
'''
)

#### Create Table

In [7]:
# Keyspace = Database
session.execute(
'''                
CREATE TABLE snu.student ( 
    year int,
    id int,
    name text, 
    dept text, 
    email text,
    PRIMARY KEY (year, id)
) WITH CLUSTERING ORDER BY (id ASC)
'''
)

In [8]:
session.execute("SELECT keyspace_name FROM system_schema.keyspaces").all()

[Row(keyspace_name='system_auth'),
 Row(keyspace_name='system_schema'),
 Row(keyspace_name='snu'),
 Row(keyspace_name='system_distributed'),
 Row(keyspace_name='system'),
 Row(keyspace_name='system_traces')]

In [9]:
session.execute("SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'snu'").all()

[Row(table_name='student')]

In [10]:
session.execute(
'''
SELECT * FROM system_schema.columns
WHERE keyspace_name = 'snu' AND table_name = 'student';
'''
).all()

[Row(keyspace_name='snu', table_name='student', column_name='dept', clustering_order='none', column_name_bytes=b'dept', kind='regular', position=-1, type='text'),
 Row(keyspace_name='snu', table_name='student', column_name='email', clustering_order='none', column_name_bytes=b'email', kind='regular', position=-1, type='text'),
 Row(keyspace_name='snu', table_name='student', column_name='id', clustering_order='asc', column_name_bytes=b'id', kind='clustering', position=0, type='int'),
 Row(keyspace_name='snu', table_name='student', column_name='name', clustering_order='none', column_name_bytes=b'name', kind='regular', position=-1, type='text'),
 Row(keyspace_name='snu', table_name='student', column_name='year', clustering_order='none', column_name_bytes=b'year', kind='partition_key', position=0, type='int')]

#### Insert records into the table

In [11]:
# Inserts need primary keys
session.execute(
'''
INSERT INTO snu.student (year, id, name, dept, email)
  VALUES(2019, 55555, 'Jun Ha Chun', 'ECE', 'nikriz@snu.ac.kr');
'''
)
session.execute(
'''
INSERT INTO snu.student (year, id, name, dept, email)
  VALUES(2020, 44444, 'Ji Soo Jang', 'DS', 'simonjisu@snu.ac.kr');
'''
)
session.execute(
'''
INSERT INTO snu.student (year, id, name, dept, email)
  VALUES(2020, 11111, 'Kim Kim Kim', 'ECE', 'kim@snu.ac.kr');
'''
)
session.execute(
'''
INSERT INTO snu.student (year, id, name, dept, email)
  VALUES(2019, 22222, 'Lee Lee Lee', 'DS', 'lee@snu.ac.kr');
'''
)
session.execute(
'''
INSERT INTO snu.student (year, id, name, dept, email)
  VALUES(2020, 33333, 'Park Park Park', 'DS', 'park@snu.ac.kr');
'''
)

In [12]:
session.execute("SELECT * FROM snu.student").all()

[Row(year=2019, id=22222, dept='DS', email='lee@snu.ac.kr', name='Lee Lee Lee'),
 Row(year=2019, id=55555, dept='ECE', email='nikriz@snu.ac.kr', name='Jun Ha Chun'),
 Row(year=2020, id=11111, dept='ECE', email='kim@snu.ac.kr', name='Kim Kim Kim'),
 Row(year=2020, id=33333, dept='DS', email='park@snu.ac.kr', name='Park Park Park'),
 Row(year=2020, id=44444, dept='DS', email='simonjisu@snu.ac.kr', name='Ji Soo Jang')]

In [13]:
session.execute(
'''
UPDATE snu.student
SET email = 'junha@snu.ac.kr'
WHERE year=2019 AND id = 55555 IF EXISTS;
'''
)

In [14]:
session.execute("SELECT * FROM snu.student").all()

[Row(year=2019, id=22222, dept='DS', email='lee@snu.ac.kr', name='Lee Lee Lee'),
 Row(year=2019, id=55555, dept='ECE', email='junha@snu.ac.kr', name='Jun Ha Chun'),
 Row(year=2020, id=11111, dept='ECE', email='kim@snu.ac.kr', name='Kim Kim Kim'),
 Row(year=2020, id=33333, dept='DS', email='park@snu.ac.kr', name='Park Park Park'),
 Row(year=2020, id=44444, dept='DS', email='simonjisu@snu.ac.kr', name='Ji Soo Jang')]

#### Delete record from the table

In [15]:
session.execute(
'''
INSERT INTO snu.student (year, id, name, dept, email)
  VALUES(9999, 9999, 'Not human', 'X', 'X@snu.ac.kr');
'''
)

In [16]:
session.execute("DELETE FROM snu.student WHERE year = 9999")

In [17]:
session.execute("SELECT * FROM snu.student").all()

[Row(year=2019, id=22222, dept='DS', email='lee@snu.ac.kr', name='Lee Lee Lee'),
 Row(year=2019, id=55555, dept='ECE', email='junha@snu.ac.kr', name='Jun Ha Chun'),
 Row(year=2020, id=11111, dept='ECE', email='kim@snu.ac.kr', name='Kim Kim Kim'),
 Row(year=2020, id=33333, dept='DS', email='park@snu.ac.kr', name='Park Park Park'),
 Row(year=2020, id=44444, dept='DS', email='simonjisu@snu.ac.kr', name='Ji Soo Jang')]

#### Empty, Drop the table / Drop the keyspace

In [ ]:
#session.execute("TRUNCATE TABLE snu.student")

In [ ]:
#session.execute("DROP TABLE snu.student")

In [ ]:
#session.execute("DROP KEYSPACE snu")

![title](./img/Cassandra_Tutorials_06.png)

![title](./img/Cassandra_Tutorials_07.png)

### Primary keys

- You can't query by range with partition key.

- You are not recommended to query only by cluster key.

- You are very not recommended to query by non primary key.

- You must design query → table / primary key

In [18]:
session.execute("SELECT * FROM snu.student WHERE year > 2019").all()

InvalidRequest: Error from server: code=2200 [Invalid query] message="Only EQ and IN relation are supported on the partition key (unless you use the token() function)"

In [21]:
session.execute("SELECT * FROM snu.student WHERE id = 20000").all()

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

In [19]:
session.execute("SELECT * FROM snu.student WHERE id = 20000 ALLOW FILTERING").all() 

[]

In [20]:
session.execute("SELECT * FROM snu.student WHERE year = 2019 AND id = 20000").all()

[]

In [22]:
session.execute("SELECT * FROM snu.student WHERE year = 2019 AND id > 20000").all()

[Row(year=2019, id=22222, dept='DS', email='lee@snu.ac.kr', name='Lee Lee Lee'),
 Row(year=2019, id=55555, dept='ECE', email='junha@snu.ac.kr', name='Jun Ha Chun')]

### TODO
- Replication factor
- Consistency
- Async
- Compaction / sstable / rocksdb 
- BATCH
- Materialized view

### Distributed Setting
- Docker
- Astra

In [23]:
from cassandra.auth import PlainTextAuthProvider
import json

key = {}
with open('key.txt') as f:
    key = json.load(f)

cloud_config= {
        'secure_connect_bundle': './secure-connect-cassandra-tutorial.zip'
}
auth_provider = PlainTextAuthProvider(key['clientID'], key['clientSecret'])
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

session.execute("select release_version from system.local").one()

Row(release_version='4.0.0.6816')

In [24]:
session.execute("SELECT * FROM system.peers").all()

[Row(peer='172.24.35.2', data_center='asia-south1', dse_version=None, graph=None, host_id=UUID('b89edb89-3de0-4228-8046-499e8a4a2e75'), jmx_port=7199, native_transport_address='172.24.35.2', native_transport_port=29042, native_transport_port_ssl=None, preferred_ip='172.24.35.2', rack='rack1', release_version='4.0.0.6816', rpc_address='172.24.35.2', schema_version=UUID('aaa07454-fa93-3d2d-b7b4-c5da9f2f87fd'), server_id=None, storage_port=7000, storage_port_ssl=7001, tokens=SortedSet(['-3973216027162921237', '-4824212584856678155', '-5641161138316102174', '-6076354814903314155', '-7880102670609790399', '4513513936355413163', '666993213066548225', '7146936265093438770']), workload=None, workloads=None),
 Row(peer='172.24.26.2', data_center='asia-south1', dse_version=None, graph=None, host_id=UUID('36491cd1-63ba-428b-85a2-1c08c1c6a661'), jmx_port=7199, native_transport_address='172.24.26.2', native_transport_port=29042, native_transport_port_ssl=None, preferred_ip='172.24.26.2', rack='rack

### References, Documentations, Materials 

https://phoenixnap.com/kb/install-cassandra-on-windows

https://docs.datastax.com/en/cql-oss/3.3/index.html

https://cassandra.apache.org/doc/latest/cassandra/tools/cqlsh.html

https://docs.datastax.com/en/developer/python-driver/3.25/getting_started/

https://www.freecodecamp.org/news/the-apache-cassandra-beginner-tutorial/#tune-for-consistency-by-setting-up-a-strong-consistency-application
